MINI PROJECT : 2

PROBLEM STATEMENT





Implement BlackJack using python.

- The game is required to be only player vs dealer

- Skip the implementation of ‘split’ and ‘doubling’.


Here is a list of some basic rules (except splitting and doubling):

The goal of blackjack is to beat the dealer's hand without going over 21.

Face cards are worth 10. Aces are worth 1 or 11, whichever makes a better hand.

Each player starts with two cards, one of the dealer's cards is hidden until the end.

To 'Hit' is to ask for another card. To 'Stand' is to hold your total and end your turn.

If you go over 21 you bust, and the dealer wins regardless of the dealer's hand.

If you are dealt 21 from the start (Ace & 10), you got a blackjack (instant win).

Blackjack usually means you win 1.5 the amount of your bet. Depends on the casino.

Dealer will hit until his/her cards total 17 or higher i.e once the player chooses to hold (and they aren't busted yet) the dealer has to open the hidden card and keep hitting until the sum is 17 or more

If dealer gets busted, player wins


It is advised to play some games online/ read more about it before implementing.


Create a login system for the game as well:

A player can sign up:
- Ask for username and password

- Create a player table and save player details in MySQL database (password should be encrypted)

- usernames should be unique


Next, a player can log in to the system and start playing.

For every game store player’s final score, dealer’s final score, the result of the game (win or loss) and the timestamp  in a “results” table.


The player is greeted with a welcome menu on logging in, where they can select to view their scores:

A table that shows win/loss , scores of both dealer and player and date and time in DD/MM/YYYY HH:mm:ss format


The player is asked if they want to play more after each match



END OF MINI PROJECT PROBLEM STATEMENT

Answer:

In [1]:

#create sqlite db for storing user credential informations and blackjack result table 

import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def main():
    database = r"Blackjacksqlite.db"
#user details table
    sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS Users (
                                        name text PRIMARY KEY,
                                        email TEXT NOT NULL UNIQUE,
                                        password TEXT NOT NULL UNIQUE
                                    ); """
#create result table
    sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS result (
                                    id integer PRIMARY KEY,
                                    name Text NOT NULL,
                                    dealer_score integer NOT NULL,
                                    user_score integer NOT NULL,
                                    game TEXT NOT NULL,
                                    sqltime TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL          
                                );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_projects_table)

        # create tasks table
        create_table(conn, sql_create_tasks_table)
    else:
        print("Error! cannot create the database connection.")


if __name__ == '__main__':
    main()

In [1]:
import sqlite3
from sqlite3 import Error
import random
import getpass
#import sys


def login():
    user = input('User name for login: ')
    password = getpass.getpass('Password: ')
    db = sqlite3.connect('Blackjacksqlite.db')
    c = db.cursor()
    c.execute('SELECT * from Users WHERE name="%s" AND password="%s"' % (user, password))
    if c.fetchone() is not None:
        print("Welcome, ",user)
        while(1):
            isview = input('Pls press , \n 1 for viewing previous results \n 2 for play game  \n any other key to quit ')
            if isview=='1':
                print('results table')
                db = sqlite3.connect('Blackjacksqlite.db')
                c = db.cursor()
                c.execute('SELECT * from result WHERE name="%s"' % (user))
                items=c.fetchall()
                print("Sr No     dealer score      player score      win/loss                  time    ")
                for item in items:
                    
                    print("  {}          {}                  {}             {}             {}".format(item[0],item[2],item[3],item[4],item[5]))
                    
                go_back= input('press any key to go back ')
                if go_back is not None:
                    continue         
            elif isview =='2':
                return user
            else:
                print('see u next time!!')
                return None
    else:
        print("Login failed")


class Card:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value

    def __repr__(self):
        return " of ".join((self.value, self.suit))


class Deck:
    def __init__(self):
        self.cards = [Card(s, v) for s in ["Spades", "Clubs", "Hearts", "Diamonds"] for v in
                      ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]]

    def shuffle(self):
        if len(self.cards) > 1:
            random.shuffle(self.cards)

    def deal(self):
        if len(self.cards) > 1:
            return self.cards.pop(0)


class Hand:
    def __init__(self, dealer=False):
        self.dealer = dealer
        self.cards = []
        self.value = 0

    def add_card(self, card):
        self.cards.append(card)

    def calculate_value(self):
        self.value = 0
        has_ace = False
        for card in self.cards:
            if card.value.isnumeric():
                self.value += int(card.value)
            else:
                if card.value == "A":
                    has_ace = True
                    self.value += 11
                else:
                    self.value += 10

        if has_ace and self.value > 21:
            self.value -= 10

    def get_value(self):
        self.calculate_value()
        return self.value

    def display(self):
        if self.dealer:
            print("hidden")
            print(self.cards[1])
        else:
            for card in self.cards:
                print(card)
            print("Value:", self.get_value())


class Game:
    def __init__(self,user_name):
        self.user_name=user_name
        pass

    def play(self):
        playing = True
        
        while playing:
            self.deck = Deck()
            self.deck.shuffle()

            self.player_hand = Hand()
            self.dealer_hand = Hand(dealer=True)

            for i in range(2):
                self.player_hand.add_card(self.deck.deal())
                self.dealer_hand.add_card(self.deck.deal())

            print("Your hand is:")
            self.player_hand.display()
            print()
            print("Dealer's hand is:")
            self.dealer_hand.display()

            game_over = False

            while not game_over:
                player_has_blackjack, dealer_has_blackjack = self.check_for_blackjack()
                if player_has_blackjack or dealer_has_blackjack:
                    game_over = True
                    self.show_blackjack_results(player_has_blackjack, dealer_has_blackjack)
                    continue

                choice = input("Please choose [Hit / Stick] ").lower()
                while choice not in ["h", "s", "hit", "stick"]:
                    choice = input("Please enter 'hit' or 'stick' (or H/S) ").lower()
                if choice in ['hit', 'h']:
                    self.player_hand.add_card(self.deck.deal())
                    self.player_hand.display()
                    if self.player_is_over():
                        print("You have lost!")
                        player_hand_value = self.player_hand.get_value()
                        dealer_hand_value = self.dealer_hand.get_value()
                        db = sqlite3.connect('Blackjacksqlite.db')
                        c = db.cursor()
                        username=self.user_name
                        user_score=player_hand_value
                        dealer_score = dealer_hand_value
                        game='loss'
                        values=(username,dealer_score,user_score,game)
                        sql = ''' INSERT INTO result(name,dealer_score,user_score,game)
                      VALUES(?,?,?,?) '''
                        c.execute(sql, values)
                        db.commit()
                        game_over = True
                else:
                    player_hand_value = self.player_hand.get_value()
                    dealer_hand_value = self.dealer_hand.get_value()

                    print("Final Results")
                    print("Your hand:", player_hand_value)
                    print("Dealer's hand:", dealer_hand_value)

                    if player_hand_value > dealer_hand_value:
                        print("You Win!")
                        game='win'
                    elif player_hand_value == dealer_hand_value:
                        print("Tie!")
                        game='tie'
                    else:
                        print("Dealer Wins!")
                        game='loss'
                    
                    db = sqlite3.connect('Blackjacksqlite.db')
                    c = db.cursor()
                    username=self.user_name
                    user_score=player_hand_value
                    dealer_score = dealer_hand_value
                    values=(username,dealer_score,user_score,game)
                    sql = ''' INSERT INTO result(name,dealer_score,user_score,game)
                  VALUES(?,?,?,?) '''
                    c.execute(sql, values)
                    db.commit()
                   
                    game_over = True
            
            again = input("Play Again? [Y/N] ")
            while again.lower() not in ["y", "n"]:
                again = input("Please enter Y or N ")
            if again.lower() == "n":
                print("Thanks for playing!")
                playing = False
            else:
                game_over = False

    def player_is_over(self):
        return self.player_hand.get_value() > 21

    def check_for_blackjack(self):
        player = False
        dealer = False
        if self.player_hand.get_value() == 21:
            player = True
        if self.dealer_hand.get_value() == 21:
            dealer = True

        return player, dealer

    def show_blackjack_results(self, player_has_blackjack, dealer_has_blackjack):
        if player_has_blackjack and dealer_has_blackjack:
            print("Both players have blackjack! Draw!")

        elif player_has_blackjack:
            print("You have blackjack! You win!")

        elif dealer_has_blackjack:
            print("Dealer has blackjack! Dealer wins!")


if __name__ == "__main__":
    while(1):
        use= input('new user? pls press y to register Or n to login: ').lower()
        if use=='y':
            while(1):
                user = input('User name: ')
                password = getpass.getpass('Password: ')
                password2 = getpass.getpass('confirm Password: ')
                if password == password2:
                    email = input('Email: ')
                    break
                else:
                    print('password didnt match')
                    continue

            db = sqlite3.connect('Blackjacksqlite.db')
            c = db.cursor()
            values=(user,email,password)
            sql = ''' INSERT INTO Users(name,email,password)
          VALUES(?,?,?) '''
            c.execute(sql, values)
            db.commit()
            login()
            print("                 ")
            break 
        elif use == 'n':
            isuserlogin=login()
            if isuserlogin is not None:
                g = Game(isuserlogin)
                g.play()
                break
            else:
                break
        else:
            print('pls, try again with proper input')
       
            
    
    
    

new user? pls press y to register Or n to login: n
User name for login: don
Password: ········
Welcome,  don
Pls press , 
 1 for viewing previous results 
 2 for play game  
 any other key to quit s
see u next time!!
